* precision comparison of different Hamiltonian o-Pi
* Construction : sparse/dense 
* Diagonalization : sparse/dense
* Operating point : pi / 0
* Standard scQubit : high capacity & resolution
* Variation in Flux Grid : range and density

In [1]:
from DiSuQ.Torch import models
from DiSuQ.Torch.circuit import hamiltonianEnergy
import scqubits
from torch.linalg import inv
from torch import tensor,stack,lobpcg,float,log as ln
from numpy import arange,linspace,meshgrid,array,log,argsort,pi,roll,var,std,sqrt,flip,around,diag,diff
from DiSuQ.utils import plotCompare,plotHeatmap,plotBox
from DiSuQ.Torch.optimization import uniformParameters,truncNormalParameters,initializationSequential
from DiSuQ.Torch import optimization
from DiSuQ.Torch.optimization import lossDegeneracyTarget,Adam
from DiSuQ.Torch.components import indE,capE
from torch import set_num_threads
set_num_threads(64)

In [2]:
L0 = indE(1e-7); L_ = indE(5e-2); print('Inductance Bound(GHz):',L_,L0+L_)
C0 = capE(1e-16); C_ = capE(1e-10) ; print('Capacitance Bound(GHz):',C_,C0+C_)
CJ0 = capE(1e-16); CJ_ = capE(1e-10) ; print('Shunt Bound(GHz):',CJ_,CJ0+CJ_)
J0 = 250. ; J_ = 1e-3 ; print('Junction Bound(GHz):',J_,J0+J_)
# components['Jx'].J0 = J0 ; components['Jy'].J0 = J0

Inductance Bound(GHz): 3.269230258996635e-06 1.6346183987285767
Capacitance Bound(GHz): 0.00019370229307707598 193.70248677936902
Shunt Bound(GHz): 0.00019370229307707598 193.70248677936902
Junction Bound(GHz): 0.001 250.001


In [3]:
basis = {'Chi':7,'Theta':10,'Phi':50}

### circuit samples

In [4]:
sample = []

In [5]:
# D = nan flux = .5
sample.append(dict(
El = 0.28348127007484436 ,
Ej = 9.974737167358398 ,
EcJ = 0.11167890578508377,
Ec = 10.005061149597168 ))

In [6]:
# D = 3.37 flux = .5
sample.append(dict(
El = 0.506168007850647 ,
Ec = 0.0020175580866634846 ,
Ej = 1.3809669017791748 ,
EcJ = 9.573041915893555))

In [7]:
# D = 5.96 flux = .5
sample.append(dict(
El = 0.4646588861942291 ,
Ej = 11.059173583984375 ,
EcJ = 0.5688765645027161 ,
Ec = 9.976311683654785))

In [8]:
# D = 4.08 flux = .5
sample.append(dict(
El = 0.4999999701976776 ,
Ej = 0.5000043511390686 ,
EcJ = 0.005002474412322044 ,
Ec = 0.005002433434128761))

In [9]:
sample.append(dict(
    Ej = 0.678629457950592,
EcJ = 0.004500730894505978,
El = 0.015017418190836906,
Ec = 0.024113938212394714))

In [10]:
sample.append(dict(
    Ej = 0.7142705321311951,
EcJ = 0.004358768463134766,
El = 0.04127487912774086,
Ec = 0.05171947181224823))

In [11]:
sample.append((dict(
    Ej = 0.6787948608398438,
EcJ = 0.004506761208176613,
El = 0.015344121493399143,
Ec = 0.024451106786727905)))

In [12]:
def torchDegeneracy(circ,grid_range,grid_points,phi_ext,sparse,grad):
    rep = 'R'
    basis['Phi'] = grid_points
    Ej,EcJ,Ec,El = circ['Ej'],circ['EcJ'],circ['Ec'],circ['El']
    circuit = models.zeroPi(basis,Ej=Ej,Ec=Ec,El=El,EcJ=EcJ,
                            sparse=sparse,symmetry=True,
                            _L_=(L_,L0),_C_=(C_,C0),_J_=(J_,J0),_CJ_=(CJ_,CJ0),
                            ridge=True,flux0=pi*12)
    H_J = circuit.ridgeJosephson(circuit)
    H_LC = circuit.ridgeHamiltonianLC(circuit)
    spectrum = circuit.circuitEnergy(H_LC,H_J,external_fluxes={'Lx':tensor(phi_ext)},grad=grad)
    G,I,II = spectrum[:3]+2*Ej
    if grad:
        D = ln((II-G)/(I-G))/ln(tensor(10.))
        D = D.detach().item()
    else:
        D = log((II-G)/(I-G))/log(10)
    return D

In [13]:
def scQubitDegeneracy(circ,grid_range,grid_points,phi_ext):
    Ej,EcJ,Ec,El = circ['Ej'],circ['EcJ'],circ['Ec'],circ['El']
    EcS = 1/(1/Ec+1/EcJ)
    flux_grid = scqubits.Grid1d(-12*pi, 12*pi, grid_points)
    zero_pi = scqubits.FullZeroPi(grid = flux_grid,
                       EJ   = Ej,
                       EL   = El,
                       ECJ  = EcJ, EC = None, ECS = EcS,
                       #EC  = 1/(1/EcS-1/EcJ),
                       ng   = 0, dEJ  = 0.0,
                         dCJ  = 0.0,
                         dEL  = 0.0,
                         dC   = 0.0,
                       flux = phi_ext,
                       ncut = basis['Theta'], zeropi_cutoff = 5,
                         zeta_cutoff = basis['Chi'])
    G,I,II = zero_pi.eigenvals(3)
    D = log((II-G)/(I-G))/log(10)
    return D

In [14]:
def sampleSimulation(grid,points,phi_ext):
    ST,SN,DT,DN,SC = [],[],[],[],[]
    for circ in sample:
        ST.append(torchDegeneracy(circ,grid,points,phi_ext,True,True)) # lobpcg
        SN.append(torchDegeneracy(circ,grid,points,phi_ext,True,False))# eigsh
        DT.append(torchDegeneracy(circ,grid,points,phi_ext,False,True))# torch
        DN.append(torchDegeneracy(circ,grid,points,phi_ext,False,False))# numpy
        SC.append(scQubitDegeneracy(circ,grid,points,phi_ext)) # scQubits
    return array(ST),array(SN),array(DT),array(DN),array(SC)

In [15]:
points = [10,20,40,50,60,80]
grid = [2*pi,6*pi,10*pi,16*pi,24*pi]

## Phi ext = pi

In [16]:
Points = []
for point in points:
    print(point,'**********')
    ST,SN,DT,DN,SC = sampleSimulation(6*pi,point,phi_ext=.5)
    Points.append((ST,SN,DT,DN,SC))

10 **********


/Users/chishti/JupyterHub/DiSuQ/Torch/sparse.py:179: UserWarning:

Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at ../aten/src/ATen/SparseCsrTensorImpl.cpp:54.)

/Users/chishti/JupyterHub/DiSuQ/Torch/circuit.py:741: UserWarning:

Casting complex values to real discards the imaginary part (Triggered internally at ../aten/src/ATen/native/Copy.cpp:276.)

/tmp/ipykernel_83614/773778711.py:17: RuntimeWarning:

divide by zero encountered in scalar divide



20 **********
40 **********
50 **********
60 **********
80 **********


In [20]:
PointsPi = array(Points)

In [22]:
Grid = []
for bound in grid:
    print(bound,'**********')
    ST,SN,DT,DN,SC = sampleSimulation(bound,60,phi_ext=.5)
    Grid.append((ST,SN,DT,DN,SC))

6.283185307179586 **********


/tmp/ipykernel_83614/773778711.py:17: RuntimeWarning:

divide by zero encountered in scalar divide



18.84955592153876 **********
31.41592653589793 **********
50.26548245743669 **********
75.39822368615503 **********


In [23]:
GridPi = array(Grid)

## Phi ext = Zero

In [24]:
Points = []
for point in points:
    print(point,'**********')
    ST,SN,DT,DN,SC = sampleSimulation(6*pi,point,phi_ext=0.)
    Points.append((ST,SN,DT,DN,SC))

10 **********
20 **********
40 **********
50 **********
60 **********
80 **********


In [25]:
PointsZero = array(Points)

In [26]:
Grid = []
for bound in grid:
    print(bound,'**********')
    ST,SN,DT,DN,SC = sampleSimulation(bound,60,phi_ext=.0)
    Grid.append((ST,SN,DT,DN,SC))

6.283185307179586 **********
18.84955592153876 **********
31.41592653589793 **********
50.26548245743669 **********
75.39822368615503 **********


In [27]:
GridZero = array(Grid)

## Analysis

In [35]:
circ = ['A','B','C','D','E','F','G']
method = ['ST','SN','DT','DN','SC']
len(sample),len(points),len(grid)

(7, 6, 5)

## Grid Points : Convergence

In [41]:
for case in range(len(circ)):
    plot = dict(zip(method,[PointsPi[:,index,case] for index in range(len(method))]))
    plotCompare(points,plot,'Circuit '+circ[case]+' Pi: Construction-Diagonalization','grid points','degeneracy')

* grid = 50 is a minima 
* Significant difference between : SN==DT & ST==SC
* circuit C has least variation
* DN is unstable

In [42]:
for case in range(len(circ)):
    plot = dict(zip(method,[PointsZero[:,index,case] for index in range(len(method))]))
    plotCompare(points,plot,'Circuit '+circ[case]+' Zero: Construction-Diagonalization','grid points','degeneracy')

* all methods are significantly similar
* circuit D shows gradual convergence
* grid = 50 is not necessarily a minima
* degeneracy falls with higher grid points

## Grid Range : Capacity

In [44]:
for case in range(len(circ)):
    plot = dict(zip(method,[GridPi[:,index,case] for index in range(len(method))]))
    plotCompare(grid,plot,'Circuit '+circ[case]+' Pi: Construction-Diagonalization','grid range','degeneracy')

* circuit C has least variation among methods
* Significant difference between : ST==SC & SN==DT
* ST==SC display least variation across grid range

In [45]:
for case in range(len(circ)):
    plot = dict(zip(method,[GridZero[:,index,case] for index in range(len(method))]))
    plotCompare(grid,plot,'Circuit '+circ[case]+' Zero: Construction-Diagonalization','grid range','degeneracy')

* circuit G,E display highest variation among methods
* All methods are significantly similar